In [8]:
!pip install pymupdf pandas

In [9]:
import fitz
import pandas as pd
from google.colab import files

In [11]:
def extract_graph_data_from_pdf(pdf_path, output_xlsx):
    """
    Extract graph data by analyzing PDF vector elements (paths, lines)
    instead of processing images directly.
    """
    doc = fitz.open(pdf_path)
    all_data = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        paths = page.get_drawings()
        graph_elements = []
        for path in paths:
            for item in path["items"]:
                if item[0] in ("l", "c"):
                    graph_elements.append(item[1:])

        if len(graph_elements) > 10:
            print(f"📊 Possible graph found on page {page_num+1}")
            points = []
            for elem in graph_elements:
                if isinstance(elem[0], (list, tuple)):
                    points.extend(elem[0])
                else:
                    points.append(elem[:2])
                    points.append(elem[2:])

            if points:
                df = pd.DataFrame(points, columns=["X", "Y"])
                df["Page"] = page_num + 1
                df["Graph_ID"] = f"Graph_{len(all_data)+1}"
                all_data.append(df)

    if all_data:
        result = pd.concat(all_data)
        result.to_excel(output_xlsx, index=False)
        print(f"✅ Saved data from {len(all_data)} graphs to {output_xlsx}")
        return output_xlsx
    else:
        print("❌ No graph data found in the PDF")
        return None
print("📤 Please upload your PDF file")
uploaded = files.upload()
pdf_file = next(iter(uploaded))
output_file = pdf_file.replace('.pdf', '_graph_data.xlsx')

extract_graph_data_from_pdf(pdf_file, output_file)
files.download(output_file)

📤 Please upload your PDF file


Saving 1686b40981c6d0FADA releases June 2025 Vehicle Retail Data.pdf to 1686b40981c6d0FADA releases June 2025 Vehicle Retail Data (2).pdf
📊 Possible graph found on page 5
📊 Possible graph found on page 6
✅ Saved data from 2 graphs to 1686b40981c6d0FADA releases June 2025 Vehicle Retail Data (2)_graph_data.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>